In [1]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
! mkdir ~/.kaggle

In [4]:
!cp /content/drive/MyDrive/kaggle_api/kaggle.json ~/.kaggle/kaggle.json

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
! kaggle datasets download cherylalmeira/facemask

 58% 25.0M/43.4M [00:00<00:00, 62.5MB/s]
100% 43.4M/43.4M [00:00<00:00, 89.4MB/s]


In [7]:
! unzip facemask.zip

Archive:  facemask.zip
  inflating: Dataset/Test/WithMask/1.jpg  
  inflating: Dataset/Test/WithMask/10.jpg  
  inflating: Dataset/Test/WithMask/2.jpg  
  inflating: Dataset/Test/WithMask/3.jpg  
  inflating: Dataset/Test/WithMask/4.jpg  
  inflating: Dataset/Test/WithMask/5.JPG  
  inflating: Dataset/Test/WithMask/6.JPG  
  inflating: Dataset/Test/WithMask/7.jpg  
  inflating: Dataset/Test/WithMask/8.jpg  
  inflating: Dataset/Test/WithMask/9.jpg  
  inflating: Dataset/Test/WithoutMask/1.jpg  
  inflating: Dataset/Test/WithoutMask/10.JPG  
  inflating: Dataset/Test/WithoutMask/2.JPG  
  inflating: Dataset/Test/WithoutMask/3.jpg  
  inflating: Dataset/Test/WithoutMask/4.jpg  
  inflating: Dataset/Test/WithoutMask/5.jpg  
  inflating: Dataset/Test/WithoutMask/6.jpg  
  inflating: Dataset/Test/WithoutMask/7.jpg  
  inflating: Dataset/Test/WithoutMask/8.jpg  
  inflating: Dataset/Test/WithoutMask/9.jpg  
  inflating: Dataset/Train/WithMask/1.jpg  
  inflating: Dataset/Train/WithMask/10.jp

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os

In [9]:
INIT_LR = 1e-4
epochs = 20
BS = 32

In [13]:
DIRECTORY = r"/content/Dataset/Train"
CATEGORIES = ["WithMask", "WithoutMask"]

In [14]:
data = []
labels = []

In [15]:
for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        image = load_img(img_path, target_size=(224, 224))
        image = img_to_array(image)
        image = preprocess_input(image)

        data.append(image)
        labels.append(category)

In [16]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

data = np.array(data, dtype="float32")
labels = np.array(labels)

In [17]:

(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.20, stratify=labels, random_state=42)

In [18]:
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

In [19]:
baseModel = MobileNetV2(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

9406464/9406464 [==============================] - 0s 0us/step


In [20]:
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

In [21]:
model = Model(inputs=baseModel.input, outputs=headModel)

In [22]:
for layer in baseModel.layers:
    layer.trainable = False

opt = Adam(lr=INIT_LR, decay=INIT_LR / epochs)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [23]:
H = model.fit(
    aug.flow(trainX, trainY, batch_size=BS),
    steps_per_epoch=len(trainX) // BS,
    validation_data=(testX, testY),
    validation_steps=len(testX) // BS,
    epochs=epochs)

Epoch 1/20
4/4 [==============================] - 11s 2s/step - loss: 0.7815 - accuracy: 0.5625 - val_loss: 0.6749 - val_accuracy: 0.6667
Epoch 2/20
4/4 [==============================] - 7s 2s/step - loss: 0.6425 - accuracy: 0.6392 - val_loss: 0.6259 - val_accuracy: 0.6364
Epoch 3/20
4/4 [==============================] - 6s 2s/step - loss: 0.6190 - accuracy: 0.6562 - val_loss: 0.5859 - val_accuracy: 0.7576
Epoch 4/20
4/4 [==============================] - 5s 1s/step - loss: 0.6361 - accuracy: 0.6701 - val_loss: 0.5494 - val_accuracy: 0.8485
Epoch 5/20
4/4 [==============================] - 6s 2s/step - loss: 0.5493 - accuracy: 0.7734 - val_loss: 0.5219 - val_accuracy: 0.8485
Epoch 6/20
4/4 [==============================] - 6s 2s/step - loss: 0.5021 - accuracy: 0.8047 - val_loss: 0.4941 - val_accuracy: 0.8788
Epoch 7/20
4/4 [==============================] - 5s 1s/step - loss: 0.4882 - accuracy: 0.8351 - val_loss: 0.4690 - val_accuracy: 0.8485
Epoch 8/20
4/4 [========================

In [24]:
predIdxs = model.predict(testX, batch_size=BS)

2/2 [==============================] - 3s 66ms/step


In [25]:
predIdxs = np.argmax(predIdxs, axis=1)

In [26]:
print(classification_report(testY.argmax(axis=1), predIdxs, target_names=lb.classes_))

              precision    recall  f1-score   support

    WithMask       0.94      1.00      0.97        16
 WithoutMask       1.00      0.94      0.97        17

    accuracy                           0.97        33
   macro avg       0.97      0.97      0.97        33
weighted avg       0.97      0.97      0.97        33



In [28]:
from tensorflow.keras.models import save_model, load_model

In [29]:
save_model(model, filepath='/content/drive/MyDrive/kaggle_api/mask_detectors.model', save_format="h5")

In [27]:
model.save("mask_detectors.model", save_format="h5")